In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils import class_weight
from tensorflow.keras.applications.efficientnet import preprocess_input
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve, recall_score
from tensorflow.keras.models import Model

In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split

# paths for originaldataset directories
data_dir = r'Full Dataset'  
tb_dir = os.path.join(data_dir, 'Tuberculosis')
normal_dir = os.path.join(data_dir, 'Normal')

# Get all file paths
tb_files = [os.path.join(tb_dir, f) for f in os.listdir(tb_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
normal_files = [os.path.join(normal_dir, f) for f in os.listdir(normal_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]

# Create labels
tb_labels = [1] * len(tb_files)
normal_labels = [0] * len(normal_files)

# Combine
all_files = tb_files + normal_files
all_labels = tb_labels + normal_labels

# Split into train (70%), validation (15%), test (15%)
train_files, temp_files, train_labels, temp_labels = train_test_split(
    all_files, all_labels, test_size=0.3, random_state=42, stratify=all_labels
)

val_files, test_files, val_labels, test_labels = train_test_split(
    temp_files, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
)

# Create folder structure
train_dir = os.path.join(data_dir, 'Train')
val_dir = os.path.join(data_dir, 'Validation')
test_dir = os.path.join(data_dir, 'Test')

for split_dir in [train_dir, val_dir, test_dir]:
    os.makedirs(os.path.join(split_dir, 'Tuberculosis'), exist_ok=True)
    os.makedirs(os.path.join(split_dir, 'Normal'), exist_ok=True)

# Copy files to respective folders
def copy_files(file_list, label_list, destination_dir):
    for file_path, label in zip(file_list, label_list):
        class_name = 'Tuberculosis' if label == 1 else 'Normal'
        dest_path = os.path.join(destination_dir, class_name, os.path.basename(file_path))
        shutil.copy2(file_path, dest_path)

copy_files(train_files, train_labels, train_dir)
copy_files(val_files, val_labels, val_dir)
copy_files(test_files, test_labels, test_dir)

print(f"✓ Created Train folder with {len(train_files)} images")
print(f"✓ Created Validation folder with {len(val_files)} images")
print(f"✓ Created Test folder with {len(test_files)} images")

✓ Created Train folder with 5045 images
✓ Created Validation folder with 1081 images
✓ Created Test folder with 1082 images


In [3]:
# Data set for model
train_dir = r'Dataset/Train'
val_dir = r'Dataset/Validation'
test_dir = r'Dataset/Test'

In [4]:
# Image settings
image_size = (224, 224)  
batch_size = 32

In [5]:
# Data generators
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, 
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode='nearest'
)
val_test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input  
)

In [6]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

Found 980 images belonging to 2 classes.


Found 210 images belonging to 2 classes.
Found 210 images belonging to 2 classes.


In [7]:
# Compute class weights 
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))

In [8]:
# Build model: EfficientNetB0
base_model = tf.keras.applications.EfficientNetB0(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base

In [9]:
# After initial fit and before fine-tuning:
base_model.trainable = True
# Freeze only the first N layers (experiment N≈100)
for layer in base_model.layers[:100]:
    layer.trainable = False

In [10]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

In [11]:
model = Model(inputs=base_model.input, outputs=output)


In [12]:
model.compile(optimizer=Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy'])


In [13]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-8)

In [14]:
# Training
steps_per_epoch = max(1, len(train_generator))
validation_steps = max(1, len(val_generator))

history = model.fit(
    train_generator,
    validation_data=val_generator,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    epochs=50,
    callbacks=[early_stopping, reduce_lr],
    class_weight=class_weights
)

Epoch 1/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 58s 1s/step - accuracy: 0.7684 - loss: 0.5023 - val_accuracy: 0.7190 - val_loss: 0.4740 - learning_rate: 1.0000e-04
Epoch 2/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - accuracy: 0.9367 - loss: 0.2215 - val_accuracy: 0.8762 - val_loss: 0.2897 - learning_rate: 1.0000e-04
Epoch 3/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.9500 - loss: 0.1360 - val_accuracy: 0.8476 - val_loss: 0.3428 - learning_rate: 1.0000e-04
Epoch 4/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.9673 - loss: 0.0982 - val_accuracy: 0.9143 - val_loss: 0.2245 - learning_rate: 1.0000e-04
Epoch 5/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.9806 - loss: 0.0681 - val_accuracy: 0.9381 - val_loss: 0.1947 - learning_rate: 1.0000e-04
Epoch 6/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 37s 1s/step - accuracy: 0.9857 - loss: 0.0507 - val_accuracy: 0.9667 - val_loss: 0.1085 - learning_rate: 1.0000e-04
Epoch 7/50
31/31 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.9908 - loss:

In [15]:
# Fine-tuning 
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False
    
model.compile(optimizer=Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
fine_tune_history = model.fit(
    train_generator,
    validation_data=val_generator,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    epochs=30,
    callbacks=[early_stopping, reduce_lr],
    class_weight=class_weights
)

Epoch 1/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 34s 807ms/step - accuracy: 0.9929 - loss: 0.0178 - val_accuracy: 0.9857 - val_loss: 0.0344 - learning_rate: 1.0000e-05
Epoch 2/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 23s 755ms/step - accuracy: 0.9980 - loss: 0.0118 - val_accuracy: 0.9857 - val_loss: 0.0364 - learning_rate: 1.0000e-05
Epoch 3/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 24s 770ms/step - accuracy: 0.9959 - loss: 0.0111 - val_accuracy: 0.9857 - val_loss: 0.0378 - learning_rate: 1.0000e-05
Epoch 4/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 24s 760ms/step - accuracy: 0.9969 - loss: 0.0094 - val_accuracy: 0.9857 - val_loss: 0.0379 - learning_rate: 1.0000e-05
Epoch 5/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 24s 757ms/step - accuracy: 0.9980 - loss: 0.0068 - val_accuracy: 0.9857 - val_loss: 0.0372 - learning_rate: 1.0000e-05
Epoch 6/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 23s 751ms/step - accuracy: 0.9980 - loss: 0.0104 - val_accuracy: 0.9857 - val_loss: 0.0377 - learning_rate: 5.0000e-06
Epoch 7/30
31/31 ━━━━━━━━━━━━━━━━━━━━ 23s 745ms/step - acc

In [17]:
# Save
model.save("new_efficientnet_model_fixed.h5")

In [18]:
# Evaluate
test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))
print(f"Test Accuracy: {test_acc:.4f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 343ms/step - accuracy: 0.9952 - loss: 0.0482
Test Accuracy: 0.9952


In [19]:
# Predictions
y_true = test_generator.classes
y_pred_prob = model.predict(test_generator)
y_pred = (y_pred_prob > 0.5).astype("int32")

7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 613ms/step


In [21]:
def focal_loss(alpha=0.25, gamma=2.0):
    def loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        bce = tf.keras.backend.binary_crossentropy(y_true, y_pred)
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        alpha_factor = y_true * alpha + (1 - y_true) * (1 - alpha)
        mod_factor = tf.keras.backend.pow((1 - p_t), gamma)
        return tf.keras.backend.mean(alpha_factor * mod_factor * bce)
    return loss
model.compile(
    optimizer=Adam(1e-5),          # lower LR for fine-tuning
    loss=focal_loss(alpha=0.25, gamma=2.0),
    metrics=['accuracy']
)
print("\nClassification Report:\n", classification_report(y_true, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
print("AUC Score:", roc_auc_score(y_true, y_pred_prob))


Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      1.00       105
           1       1.00      0.99      1.00       105

    accuracy                           1.00       210
   macro avg       1.00      1.00      1.00       210
weighted avg       1.00      1.00      1.00       210

Confusion Matrix:
 [[105   0]
 [  1 104]]
AUC Score: 0.9999092970521543
